In [1]:
import pystokes 
import numpy as np, matplotlib.pyplot as plt

from IPython.display import Video
import os
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D

In [19]:
# particle radius, self-propulsion speed, number and fluid viscosity
b, vs, N, eta = 1.0,2, 8, 0.1


#initialise
r = pystokes.utils.initialCondition(N)  # initial random distribution of positions
p = np.zeros(3*N); 
p[2*N:3*N] = -0.4   # initial orientation of the colloids

In [20]:

def rhs(rp):
    """
    right hand side of the rigid body motion equation
    rp: is the array of position and orientations of the colloids
    returns the \dot{rp} so that rp can be updated using an integrator
    """
    # assign fresh values at each time step
    r = rp[0:3*N];   p = rp[3*N:6*N]
    norm_p = np.dot(p,p)
    p = p/np.sqrt(norm_p)
    F, v, o,T = np.zeros(3*N), np.zeros(3*N), np.zeros(3*N),np.zeros(3*N)
    
    force.lennardJonesWall(F, r, lje=0.01, ljr=5, wlje=1.2, wljr=3.4)
    torque.magnetic(T,p,m0=1,Bx=0, By=1, Bz=-0)
    #torque.bottomHeaviness(T,p,bh=1.0)
    rbm.mobilityTT(v, r, F)  
    rbm.mobilityRR(o,r,T)
    #rbm.mobilityTR(v,r,T)
    #rbm.mobilityRT(o,r,F)
    
    l2 = pystokes.utils.irreducibleTensors(2, p,Y0=1)
    l3 = pystokes.utils.irreducibleTensors(3, p)
    v3t = vs*p;
    rbm.propulsionT2s(v,r,l2)
    #rbm.propulsionR2s(o,r,l2)
    #rbm.propulsionR4a(o, r, l3)
    rbm.propulsionT3t(v, r, v);
    v = v+v3t;
    return np.concatenate( (v,o) )

rbm   = pystokes.wallBounded.Rbm(radius=b, particles=N, viscosity=eta)
force = pystokes.forceFields.Forces(particles=N)
torque = pystokes.forceFields.Torques(particles=N)

* The torque on microparticle in an external magnetic field ${\bf B}$  is ${\bf T} = {\bf m\times B}$. 
* We also note        We assume $m = m_0 \bf p$

* We choose p to be along -ve z-direction initially and apply a magnetic field along y direction.

In [21]:
T= np.zeros(3*N)
torque.magnetic(T,p,m0=1,Bx=0, By=1, Bz=-0)
T

array([0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ])

In [22]:
# simulate the resulting system
Tf, Nts = 120, 100
pystokes.utils.simulate(np.concatenate((r,p)), Tf,Nts,rhs, filename='swarms')

# plot the data at specific time instants
#pystokes.utils.plotConfigs(t=[1,40,60,80,100], ms=80, tau=1, filename='swarms')# -*- coding: utf-8 -*-
from scipy.io import loadmat
result=loadmat('swarms')
traj	  = result['X']

from statistics import mean
py=traj[0:Nts,4*N:5*N]
pymean=mean(py[Nts-1])
px=traj[0:Nts,3*N:4*N]
pxmean=mean(px[Nts-1])
pz=traj[0:Nts,5*N:6*N]
pzmean=mean(pz[Nts-1])
alpha=np.arctan(pymean/pzmean)*180/np.pi

## Plot data and make a movie

In [23]:
for i in range(Nts):
    dx=traj[i+1,0:N]-traj[i,0:N]
    dy=traj[i+1,N:2*N]-traj[i,N:2*N]
    dz=traj[i+1,2*N:3*N]-traj[i,2*N:3*N]
    speed=10*np.sqrt(dx**2+dy**2+dz**2)
    #fig,ax=plt.subplots()
    #plt.scatter(traj[i,0:N],traj[i,N:2*N],s=40,c=speed, cmap='inferno')
    #plt.xlim(-50,50)
    #plt.ylim(-50,50)
    from mpl_toolkits.mplot3d import Axes3D

    fig = plt.figure(); ax = fig.add_subplot(111, projection='3d')

    ax.quiver(traj[i,0:N], traj[i,N:2*N], traj[i,2*N:3*N], traj[i,3*N:4*N], traj[i,4*N:5*N], traj[i,5*N:6*N],length=6, arrow_length_ratio=.65)
    ax.set_xlim(-20,20)
    ax.set_ylim(-20,20)
    ax.set_zlim(0,10)
    ax.set_xlabel('x/b')
    ax.set_ylabel('y/b')
    ax.set_zlabel('z/b')

    plt.savefig('data//image-%05d_.png'%(i)); plt.clf();  plt.close()
    fig.clear()
    plt.close(fig)

In [24]:
%%capture
!ffmpeg  -y -framerate 10 -pattern_type glob -i "data//image-*.png" output.mp4  

In [25]:
Video('output.mp4')   